In [ ]:
#|default_exp coinbase

In [ ]:
#|hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Coinbase

Manages the coinbase data download and processing. It is important to note that the Coinbase API might change from time to time, and the functions might need to be updated. You can find the documentation here: https://docs.cdp.coinbase.com/exchange/reference/exchangerestapi_getproductcandles.

Notice also that there are limits of what one can download. See the documentation for more details. In particular: https://docs.cdp.coinbase.com/exchange/docs/rest-rate-limits

** Finally, datetime columns are in UTC. **


In [ ]:
#| export

from datetime import datetime
import pandas as pd
import requests
import os
import datetime as dt


In [ ]:
#| export

def retrieve_coinbase_price(pair="BTC-USD", time_interval=3600,
                          end_date=datetime.now().strftime('%Y-%m-%dT%H:%M:%SZ'),
                          start_date=(datetime.now()-pd.Timedelta(days=2)).strftime('%Y-%m-%dT%H:%M:%SZ')):
    """
    Retrieves historical price data from Coinbase for a given trading pair and time interval.

    Makes a GET request to the Coinbase candles endpoint to fetch OHLCV (Open, High, Low, Close, Volume) 
    data for the specified trading pair and time period.

    Args:
        pair (str, optional): Trading pair symbol (e.g. "BTC-USD"). Defaults to "BTC-USD".
        time_interval (int, optional): Candle interval in seconds. Defaults to 3600 (1 hour).
        end_date (str, optional): End datetime in ISO 8601 format. Defaults to current UTC time.
        start_date (str, optional): Start datetime in ISO 8601 format. Defaults to 2 days before end_date.

    Returns:
        pandas.DataFrame: DataFrame containing the OHLCV data with columns:
            - datetime: Timestamp for the candle (UTC)
            - low: Lowest traded price in the interval
            - high: Highest traded price in the interval  
            - open: Opening price of the interval
            - close: Closing price of the interval
            - volume: Trading volume in the interval
            - pair: Trading pair symbol
        Returns None if the API request fails or returns no data.

    Notes:
        - Limited to 300 requests per hour by the Coinbase API
        - Maximum of 200 candles can be retrieved per request
        - Attempting to exceed these limits will result in an error and return None
        - All datetime values are in UTC timezone
    """
    url = f"https://api.exchange.coinbase.com/products/{pair}/candles?granularity={time_interval}&start={start_date}&end={end_date}"
    response = requests.get(url)
    if response.status_code != 200:
        return None
    data = response.json()
    if not data:
        return None

    column_names = ['unix', 'low', 'high', 'open', 'close', 'volume']
    df = pd.DataFrame(data, columns=column_names)
    df['datetime'] = pd.to_datetime(df['unix'], unit='s')
    df['pair'] = pair
    df = df.sort_values(by='datetime')
    # drop unix column and put date first
    df = df.drop(columns=['unix'])
    df = df[['datetime'] + [col for col in df.columns if col != 'datetime']]
    return df


In [ ]:
#| export

def coinbase_tokens():
    """
    Retrieves all available trading pairs from the Coinbase Exchange API.

    Makes a GET request to the Coinbase products endpoint to fetch information about
    all trading pairs available on the exchange.

    Returns:
        pandas.DataFrame: DataFrame containing information about all trading pairs with columns:
            - id: Trading pair ID (e.g. 'BTC-USD')
            - base_currency: The cryptocurrency being traded (e.g. 'BTC') 
            - quote_currency: The currency used for pricing (e.g. 'USD')
            - quote_increment: Minimum price increment
            - base_increment: Minimum quantity increment
            - display_name: Human readable name of the trading pair
            - status: Trading status of the pair
            And other metadata columns provided by the Coinbase API
    """
    currencies_url = "https://api.exchange.coinbase.com/products"
    currencies_response = requests.get(currencies_url)
    currencies_data = currencies_response.json()
    return pd.DataFrame(currencies_data)


In [ ]:
#| export

def coinbase_usd_tokens():
    """
    Retrieves all trading pairs from Coinbase that have USD as the quote currency.

    Returns:
        pandas.DataFrame: DataFrame containing information about USD trading pairs with columns:
            - id: Trading pair ID (e.g. 'BTC-USD')
            - base_currency: The cryptocurrency being traded (e.g. 'BTC')
            - quote_currency: Always 'USD' for this filtered dataset
            - quote_increment: Minimum price increment
            - base_increment: Minimum quantity increment
            - display_name: Human readable name of the trading pair
            - status: Trading status of the pair
            And other metadata columns provided by the Coinbase API
    """
    tokens = coinbase_tokens()
    return tokens[tokens['quote_currency']=='USD']


In [ ]:
#| export

def coinbase_price_history(pair='BTC-USD', start_date='2024-01-01', end_date='2024-05-01', time_interval=3600, max_pull=250, verbose=False):
    """
    Downloads historical price data for a cryptocurrency pair from Coinbase, handling rate limits.

    Args:
        pair (str): Trading pair symbol (e.g. 'BTC-USD'). Defaults to 'BTC-USD'.
        start_date (str): Start date in 'YYYY-MM-DD' format. Defaults to '2024-01-01'.
        end_date (str): End date in 'YYYY-MM-DD' format. Defaults to '2024-05-01'.
        time_interval (int): Time interval between candles in seconds. Defaults to 3600 (1 hour).
        max_pull (int): Maximum number of candles per API request. Defaults to 250.
        verbose (bool): If True, prints progress messages. Defaults to False.

    Returns:
        pandas.DataFrame: DataFrame containing price history with columns:
            - datetime: Timestamp of the candle
            - low: Lowest price during the interval
            - high: Highest price during the interval  
            - open: Opening price of the interval
            - close: Closing price of the interval
            - volume: Trading volume during the interval
            - pair: Trading pair symbol

    Raises:
        ValueError: If end_date is earlier than start_date

    The function handles Coinbase's API limitations by automatically splitting requests
    into smaller chunks if the date range would exceed the maximum allowed candles
    per request.
    """
    # convert the dates to datetime
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    if end_date < start_date:
        # raise an error
        raise ValueError("End date is smaller than start date")
    # calculate the number of hours between the start and end date
    n_hours = (end_date - start_date).days * 24
    if n_hours <= max_pull:
        df = retrieve_coinbase_price(pair,start_date=start_date.strftime('%Y-%m-%dT%H:%M:%SZ'),end_date=end_date.strftime('%Y-%m-%dT%H:%M:%SZ'),time_interval=time_interval)
        return df
    # create a timeline with the maximum number of dates per request
    time_line = pd.date_range(start=start_date, end=end_date, periods=n_hours//max_pull+1)
    price_list = []
    for i in range(len(time_line)-1):
        end = time_line[i+1]
        start = time_line[i]
        if verbose:
            print(f"Downloading data from {start} to {end}")
        df = retrieve_coinbase_price(pair,start_date=start.strftime('%Y-%m-%dT%H:%M:%SZ'),end_date=end.strftime('%Y-%m-%dT%H:%M:%SZ'),time_interval=time_interval)
        if df is not None:
            price_list.append(df)
    # concatenate the dataframes and sort by date and drop duplicates dates
    if price_list == []:
        print(f"{pair} is not listed.")
        return pd.DataFrame()
    df = pd.concat(price_list)
    df = df.sort_values(by='datetime')
    df = df.drop_duplicates(subset='datetime').reset_index(drop=True)
    return df

In [ ]:
#| export

def save_file(df, folder_path, file_name, type="csv"):
    """
    Save a pandas DataFrame to a file in either CSV or Parquet format.

    Args:
        df (pandas.DataFrame): The DataFrame to save
        folder_path (str): Directory path where the file will be saved
        file_name (str): Name of the file without extension
        type (str, optional): File format - either "csv" or "parquet". Defaults to "csv"

    The function saves the DataFrame to the specified path, handling the file extension automatically.
    For CSV files, the index is not saved. For Parquet files, default Parquet settings are used.
    """
    if type == "csv":
        df.to_csv(f"{folder_path}/{file_name}.csv",index=False)
    elif type == "parquet":
        df.to_parquet(f"{folder_path}/{file_name}.parquet")



In [ ]:
#| export

def coinbase_to_file(folder_path="../data/coinbase",token_list=coinbase_usd_tokens()['id'].tolist(),type="csv",
                     interval=3600,all_tokens=True,refresh_24h=False):
    """
    Downloads and maintains historical price data for Coinbase tokens, saving to files.
    
    Args:
        folder_path (str): Path where token data files will be stored. Defaults to "../data/coinbase"
        token_list (list): List of token IDs to process. Defaults to all USD trading pairs from coinbase_usd_tokens()
        type (str): File format to save data - either "csv" or "parquet". Defaults to "csv"
        interval (int): Time interval in seconds between price points. Defaults to 3600 (1 hour)
        all_tokens (bool): If True, includes any additional tokens found in the folder path. Defaults to True
        refresh_24h (bool): If True, replaces at least 24 hours. Defaults to False
    
    The function:
    - Creates the folder_path if it doesn't exist
    - Date/Time is UTC
    - For each token, checks if data file exists:
        - If exists: Loads file and appends any new data since last recorded date
        - If not exists: Downloads full history starting from 2016
    - Saves data in specified format, handling duplicates and sorting by date
    - For hourly data (interval=3600), aligns to hour boundaries
    """
    # create the folder if it does not exist, and if it exists, read the file names
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    else:
        if all_tokens:
            file_names = os.listdir(folder_path)
            # remove the file extension
            tokens_in_folder = [os.path.splitext(file_name)[0] for file_name in file_names]
            # join tokens in folder with token_list and remove the duplicates
            token_list = list(set(token_list + tokens_in_folder))
    # loop over the token list and open the file if it exists and append the new data
    for token in token_list:
        print(f"Processing {token}")
        file_name = f"{folder_path}/{token}.{type}"
        if os.path.exists(file_name):
            if type == "csv":
                df = pd.read_csv(file_name)
            elif type == "parquet":
                df = pd.read_parquet(file_name)
            else:
                raise ValueError(f"Type {type} not supported")
            # read last date in the file
            last_date = pd.to_datetime(df['datetime'].iloc[-1]).tz_localize(dt.UTC)
            today = datetime.now(tz=dt.UTC)
            #print(last_date.tz_localize(dt.UTC),today)
            first_date = today - dt.timedelta(hours=24)
            if first_date < last_date and refresh_24h:
                t_date = pd.to_datetime(df['datetime'],utc=True)
                df = df[t_date < first_date]
                last_date = first_date
            if last_date < today:
                try:
                    df_new = coinbase_price_history(pair=token,
                                                start_date=last_date.strftime('%Y-%m-%dT%H:%M:%SZ'),
                                                end_date=today.strftime('%Y-%m-%dT%H:%M:%SZ'),
                                                time_interval=interval)
                    df = pd.concat([df,df_new])
                    # drop duplicates and sort by date
                    df = df.drop_duplicates(subset='datetime').sort_values(by='datetime').reset_index(drop=True)
                    if not df.empty:
                        save_file(df,folder_path,token,type)
                except Exception as e:
                    print(f"Error adding data to {token}: {e}")
            else:
                print(f"File {token} is up to date")
        else:
            try:
                df = coinbase_price_history(pair=token,
                                        start_date='2016-01-01T00:00:00Z',
                                        end_date=datetime.now().strftime('%Y-%m-%dT%H:%M:%SZ'),
                                        time_interval=interval)
            except Exception as e:
                print(f"Error first download for {token}: {e}")
            if not df.empty:
                save_file(df,folder_path,token,type)

In [ ]:
#| export
def coinbase_data_update(folder_path="../data/coinbase",token='AAVE-USD',type="parquet",
                     interval=3600,refresh_24h=True):
    """
    Downloads new historical price data for Coinbase tokens... no saving
    
    Args:
        folder_path (str): Path where token data files will be stored. Defaults to "../data/coinbase"
        token (str): token IDs to process
        type (str): File format to save data - either "csv" or "parquet". Defaults to "parquet"
        interval (int): Time interval in seconds between price points. Defaults to 3600 (1 hour)
        refresh_24h (bool): If True, replaces at least 24 hours. Defaults to True
    
    Returns:
        df (pandas.DataFrame): DataFrame of historical price data
    
    The function:
    - Date/Time is UTC
    - For each token, checks if data file exists:
        - If exists: Loads file and appends any new data since last recorded date
        - If not exists: Downloads full history starting from 2016
    - For hourly data (interval=3600), aligns to hour boundaries
    """
    
    if not os.path.exists(folder_path):
        raise FileNotFoundError(f"Folder {folder_path} does not exist")
    
    print(f"Processing {token}")
    file_name = f"{folder_path}/{token}.{type}"
    if os.path.exists(file_name):
        if type == "csv":
            df = pd.read_csv(file_name)
        elif type == "parquet":
            df = pd.read_parquet(file_name)
        else:
            raise ValueError(f"Type {type} not supported")
        # read last date in the file
        last_date = pd.to_datetime(df['datetime'].iloc[-1]).tz_localize(dt.UTC)
        today = datetime.now(tz=dt.UTC)
        #print(last_date.tz_localize(dt.UTC),today)
        first_date = today - dt.timedelta(hours=24)
        if first_date < last_date and refresh_24h:
                t_date = pd.to_datetime(df['datetime'],utc=True)
                df = df[t_date < first_date]
                last_date = first_date
        if last_date < today:
            df_new = coinbase_price_history(pair=token,
                                            start_date=last_date.strftime('%Y-%m-%dT%H:%M:%SZ'),
                                            end_date=today.strftime('%Y-%m-%dT%H:%M:%SZ'),
                                            time_interval=interval)
            df = pd.concat([df,df_new])
                # drop duplicates and sort by date
            df = df.drop_duplicates(subset='datetime').sort_values(by='datetime').reset_index(drop=True)
        else:
            print(f"File {token} is up to date")
    else:
        df = coinbase_price_history(pair=token,
                                        start_date='2016-01-01T00:00:00Z',
                                        end_date=datetime.now().strftime('%Y-%m-%dT%H:%M:%SZ'),
                                        time_interval=interval)
    return df

In [ ]:
#| export
def read_all_files(folder_path="../data/coinbase",type="csv"):
    """Read and combine all files from a folder into a single DataFrame.
    
    Args:
        folder_path (str): Path to the folder containing the files. Defaults to "../data/coinbase".
        type (str): File type to read - either "csv" or "parquet". Defaults to "csv".
        
    Returns:
        pandas.DataFrame: Combined DataFrame containing data from all files in the folder.
        
    Raises:
        ValueError: If file type is not supported (must be "csv" or "parquet").
    """
    file_names = os.listdir(folder_path)
    df_list = []
    for file_name in file_names:
        if type == "csv":
            df = pd.read_csv(f"{folder_path}/{file_name}")
        elif type == "parquet":
            df = pd.read_parquet(f"{folder_path}/{file_name}")
        else:
            raise ValueError(f"Type {type} not supported")
        df_list.append(df)
    return pd.concat(df_list)



In [ ]:
#| export
def coinbase_price_last_day(pair='BTC-USD'):
    """
    Fetch the last day's price for a given Coinbase token.
    This fuction is a helper function around coinbase_price_history().
    
    Args:
        pair (str): Coinbase token pair, e.g., "BTC-USD". Defaults to "BTC-USD".
    
    Returns:
        pandas dataframe: Last 24 hours price data.
    """
    # fetch the data
    data = coinbase_price_history(pair=pair, 
                        start_date=(datetime.now(tz=dt.UTC) - dt.timedelta(days=1)).strftime('%Y-%m-%dT%H:%M:%SZ'), 
                        end_date=datetime.now(tz=dt.UTC).strftime('%Y-%m-%dT%H:%M:%SZ'))
    # return the last price
    return data

In [ ]:
#| export
def binance_format(df):
    """
    Convert coinbase dataframe to binance format.
    
    Args:
        df (pandas dataframe): Coinbase dataframe.
    
    Returns:
        pandas dataframe: Binance format dataframe.
    """
    df['date']  = pd.to_datetime(df['datetime'],utc=True)
    df['pair']  = df['pair'].apply(lambda x: str.lower(str.split(x,'-')[0]))
    df['Open'] = df['open']
    df['High'] = df['high']
    df['Low']  = df['low']
    df['Close'] = df['close']
    df['Volume'] = df['volume']
    return df[['date','Open','High','Low','Close','Volume','pair']]

## Examples

Load the package

In [ ]:
from token_style.coinbase import *

### List the available tokens on Coinbase

List the available tokens on Coinbase.

In [ ]:
#| eval: false
tokens = coinbase_tokens()
print(tokens)

             id base_currency quote_currency quote_increment base_increment  \
0       LRC-USD           LRC            USD          0.0001       0.000001   
1       KNC-USD           KNC            USD          0.0001            0.1   
2      TNSR-USD          TNSR            USD           0.001           0.01   
3       UNI-EUR           UNI            EUR            0.01          0.001   
4       ADA-GBP           ADA            GBP          0.0001           0.01   
..          ...           ...            ...             ...            ...   
653    ATA-USDT           ATA           USDT          0.0001            0.1   
654   BTRST-USD         BTRST            USD           0.001           0.01   
655  PIRATE-USD        PIRATE            USD          0.0001            0.1   
656     RAD-BTC           RAD            BTC       0.0000001           0.01   
657    CTX-USDT           CTX           USDT            0.01          0.001   

    display_name min_market_funds  margin_enabled  

To make it simpler, we can filter the tokens by the quote currency. In this case, we filter for USD.


In [ ]:
#| eval: false
usd_tokens = coinbase_usd_tokens()
print(usd_tokens)


             id base_currency quote_currency quote_increment base_increment  \
0       LRC-USD           LRC            USD          0.0001       0.000001   
1       KNC-USD           KNC            USD          0.0001            0.1   
2      TNSR-USD          TNSR            USD           0.001           0.01   
5       INJ-USD           INJ            USD           0.001           0.01   
8       PLA-USD           PLA            USD          0.0001            0.1   
..          ...           ...            ...             ...            ...   
648     TRB-USD           TRB            USD            0.01          0.001   
649     RAI-USD           RAI            USD            0.01           0.01   
652     GRT-USD           GRT            USD          0.0001           0.01   
654   BTRST-USD         BTRST            USD           0.001           0.01   
655  PIRATE-USD        PIRATE            USD          0.0001            0.1   

    display_name min_market_funds  margin_enabled  

### Data pull examples

Download the historical price of BTC-USD for few months at every hour. IMPORTANT: datetime is in UTC!


In [ ]:
#| eval: false
df = coinbase_price_history()
print(df)

                datetime       low      high      open     close       volume  \
0    2024-01-01 00:00:00  42261.58  42543.64  42288.58  42452.66   379.197253   
1    2024-01-01 01:00:00  42415.00  42749.99  42453.83  42594.68   396.201924   
2    2024-01-01 02:00:00  42488.03  42625.68  42594.58  42571.32   227.141166   
3    2024-01-01 03:00:00  42235.00  42581.26  42571.32  42325.11   306.005694   
4    2024-01-01 04:00:00  42200.00  42393.48  42325.10  42389.77   296.233644   
...                  ...       ...       ...       ...       ...          ...   
2900 2024-04-30 20:00:00  59087.36  60082.82  59098.18  59869.99  1655.908771   
2901 2024-04-30 21:00:00  59862.17  60326.56  59865.66  60100.36   615.103455   
2902 2024-04-30 22:00:00  60070.00  60990.48  60100.01  60526.64   727.224395   
2903 2024-04-30 23:00:00  60330.92  60878.39  60529.65  60622.10   346.354179   
2904 2024-05-01 00:00:00  60015.99  60785.49  60621.20  60173.03   499.935743   

         pair  
0     BTC-U

Download all history available in coinbase for the tokens saved in folder "../data/coinbase" 
This process can take a while specially if you are downloading many tokens the first time.


Full download with no files in the folder takes almost 5 hours. After that, it is much faster (~ 6 minutes). Clearly it will depend on your internet speed and the last time you did the download.


In [ ]:
#| eval: false
coinbase_to_file(type="parquet") # takes almost 5 hours if there no files in the folder!

Now read the file from folder "../data/coinbase":

In [ ]:
#| eval: false
import pandas as pd
df = pd.read_parquet("../data/coinbase/AAVE-USD.parquet")
print(df)

                 datetime      low     high     open    close    volume  \
0     2020-12-15 17:00:00   87.700   90.400   90.400   88.347  4289.548   
1     2020-12-15 18:00:00   85.620   88.407   88.010   86.000  3802.529   
2     2020-12-15 19:00:00   84.575   89.500   86.105   85.952  2835.124   
3     2020-12-15 20:00:00   84.828   85.952   85.952   85.599   798.670   
4     2020-12-15 21:00:00   84.744   85.893   85.599   85.208   442.464   
...                   ...      ...      ...      ...      ...       ...   
35946 2025-01-21 16:00:00  354.870  362.290  357.310  357.700  4360.988   
35947 2025-01-21 17:00:00  359.540  366.960  360.790  366.010  3854.849   
35948 2025-01-21 18:00:00  365.050  374.330  366.000  373.260  8992.844   
35949 2025-01-21 19:00:00  369.480  378.250  373.360  371.830  7655.470   
35950 2025-01-21 20:00:00  369.730  371.930  371.790  369.790   485.523   

           pair  
0      AAVE-USD  
1      AAVE-USD  
2      AAVE-USD  
3      AAVE-USD  
4      AA

Read all files in the folder into a single dataframe:


In [ ]:
#| eval: false
df = read_all_files(type="parquet")
print(df)

                 datetime       low      high      open     close  \
0     2022-10-11 17:00:00  0.372500  0.510000  0.450000  0.510000   
1     2022-10-11 18:00:00  0.427800  1.284200  0.507800  1.284200   
2     2022-10-11 19:00:00  0.900900  1.464600  1.269000  1.300000   
3     2022-10-11 20:00:00  1.140000  1.382000  1.292100  1.331900   
4     2022-10-11 21:00:00  1.268600  1.649800  1.331900  1.546400   
...                   ...       ...       ...       ...       ...   
55015 2025-01-21 16:00:00  0.466431  0.478793  0.466431  0.474741   
55016 2025-01-21 17:00:00  0.471899  0.483596  0.473259  0.483134   
55017 2025-01-21 18:00:00  0.480222  0.488367  0.482917  0.487325   
55018 2025-01-21 19:00:00  0.480000  0.486430  0.486381  0.484624   
55019 2025-01-21 20:00:00  0.484325  0.485571  0.485246  0.484831   

             volume     pair  
0      2.214310e+05   00-USD  
1      1.261266e+06   00-USD  
2      1.587872e+06   00-USD  
3      6.753579e+05   00-USD  
4      8.359198e

Download the last day's price for AAVE-USD:

In [ ]:
# eval: false
coinbase_price_last_day(pair='AAVE-USD')

,datetime,low,high,open,close,volume,pair
23,2025-01-20 21:00:00,340.42,352.36,348.39,341.27,5889.728,AAVE-USD
22,2025-01-20 22:00:00,337.83,348.73,341.27,348.57,3904.477,AAVE-USD
21,2025-01-20 23:00:00,334.91,348.98,348.65,337.09,9045.903,AAVE-USD
20,2025-01-21 00:00:00,327.27,339.04,337.07,327.87,9903.319,AAVE-USD
19,2025-01-21 01:00:00,327.23,339.84,327.96,337.34,47457.843,AAVE-USD
18,2025-01-21 02:00:00,337.00,350.57,337.21,346.02,23115.761,AAVE-USD
17,2025-01-21 03:00:00,340.55,347.55,345.93,345.53,5440.116,AAVE-USD
16,2025-01-21 04:00:00,332.82,345.80,345.80,340.28,4284.122,AAVE-USD
15,2025-01-21 05:00:00,332.54,341.14,340.24,336.69,3109.911,AAVE-USD
14,2025-01-21 06:00:00,334.67,344.57,336.69,342.67,4486.771,AAVE-USD


Update the data for AAVE-USD and convert it to Binance format (minor changes in column names and sequence). Note: the datetime column is in UTC and the row datetime represents the full hour. For example, if we run this code at 9:15:00 the row is labelled as 9:00:00 with the close price at 9:15. If we run the code at 8:59:00, the row is labelled as 8:00:00 with the close price at 8:59.

In [ ]:
#| eval: false
df = coinbase_data_update(token='AAVE-USD')
binance_df = binance_format(df)
print(binance_df)

Processing AAVE-USD
                           date     Open     High      Low    Close    Volume  \
0     2020-12-15 17:00:00+00:00   90.400   90.400   87.700   88.347  4289.548   
1     2020-12-15 18:00:00+00:00   88.010   88.407   85.620   86.000  3802.529   
2     2020-12-15 19:00:00+00:00   86.105   89.500   84.575   85.952  2835.124   
3     2020-12-15 20:00:00+00:00   85.952   85.952   84.828   85.599   798.670   
4     2020-12-15 21:00:00+00:00   85.599   85.893   84.744   85.208   442.464   
...                         ...      ...      ...      ...      ...       ...   
35946 2025-01-21 16:00:00+00:00  357.310  362.650  354.870  360.900  5480.451   
35947 2025-01-21 17:00:00+00:00  360.790  366.960  359.540  366.010  3854.849   
35948 2025-01-21 18:00:00+00:00  366.000  374.330  365.050  373.260  8992.844   
35949 2025-01-21 19:00:00+00:00  373.360  378.250  369.480  371.830  7676.252   
35950 2025-01-21 20:00:00+00:00  371.790  376.860  367.670  374.440  2297.664   

       

In [ ]:
#| hide
#import pandas as pd
#df = coinbase_data_update(token = 'BTC-USD')
#binance_df1 = binance_format(df)
#df = coinbase_data_update(token = 'ETH-USD')
#binance_df2 = binance_format(df)
#dh = pd.concat([binance_df1, binance_df2])

In [ ]:
#| hide
#dh.to_feather("coinHist.fdr")
#dg = pd.read_feather("../../manta/trading_strategy/data/priceHist.fdr")
#print(dg)
#binance_df2.tail(3)
#coinbase_price_last_day(pair='ETH-USD').tail(3)